In [416]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"# prints every statement instead of just last one

In [153]:
datasetfolder = "C:\\Users\\spunna\\Data Science\\Datasets\\OilGas\\"

In [452]:
class OilProducers:
    def __init__(self, filepath, years, removeNotReportedWells):
        self.file = filepath
        self.years = years
        self.data = pd.read_csv(self.file, usecols=['District','Year','Month','Operator','Name','No','Oil(BBL)'],
                                        dtype = {'District':str, 
                                                       'Year':int,
                                                       'Month':str,
                                                       'Operator':int,
                                                       'Name':str,
                                                       'Oil(BBL)':int,
                                                       'Casinghead(MCF)':int,
                                                       'GWGas(MCF)':int ,                                                    
                                                       'Condensate(BBL)':int})
        TotalRecords = len(self.data)
        NonZeroRecords = len(self.data[self.data['Oil(BBL)'] > 0])
        if (removeNotReportedWells):
            self.data = self.data[self.data['Oil(BBL)'] > 300]
        RecordsMoreThan300 = len(self.data)
        print('Oil Production Data for years:{}'.format(years))
        print('Total Records: {}, Non-Zero Records: {}'.format(TotalRecords, NonZeroRecords))
        print('Records > 300 bbl: {}, {}% of Non-Zero Records'.format(RecordsMoreThan300,
                                                                      math.ceil(RecordsMoreThan300*100/(NonZeroRecords))))
        
        
    def GetData(self):
        return self.data
    
    def OperatorFieldCount(self):
        TotalOperatorsAndFields = len(self.data['Name'].unique())
        TotalOperators = len(self.data[self.data['Operator'] == 1]['Name'].unique())
        TotalFields = TotalOperatorsAndFields -TotalOperators 
        self.printline('Total No. of Operators: {}, Fields: {}'.format(TotalOperators, TotalFields))
    
    def Top10Overall(self):
        self.printline('Top 10 Producers in all districts')
        return self.data.sort_values(by='Oil(BBL)', ascending = False).head(10)
    
    def TopBottom5Overall(self):
        self.printline('Top and Bottom 5 Producers in all districts')
        overall = self.data.sort_values(by='Oil(BBL)', ascending = False)
        return overall.head(5).append(overall.tail(5))
    
    def TotalByDistrict(self):
        self.printline('Total Oil produced by districts')
        return self.data.groupby(['District', 'Operator'])["Oil(BBL)"].sum().sort_values(ascending=False).reset_index()
    
    def ProducersInRange(self, fromNo, toNo, save):
        prod = self.data.groupby(['Name', 'Operator']).agg({'Name':'size', 'Oil(BBL)':'mean'}).rename(columns={'Name': 'Count'}).reset_index()
        prod_range = prod[(prod['Oil(BBL)'] > fromNo-1 ) & (prod['Oil(BBL)'] < toNo + 1 )].sort_values(by='Count', ascending=False)
        self.printline("Oil Producers between {} and {}: {}".format(fromNo, toNo, len(prod_range)))
        if (save):
            prod_range.to_csv(datasetfolder + 'ProducersInRange_350_500_' + self.years + '.csv', index=False)
        uniqueProducers = prod_range[:10]['Name'].tolist()
        self.printline('Top 10 Producers with multiple wells: \n{}'.format([i for i in uniqueProducers]))
        top10 = prod_range[prod_range['Name'].isin(uniqueProducers)]
        
        return top10
    
    def UniqueDistrictMonthYear(self):
        print('Unique District Month Year combination')
        return self.data.groupby(['District', 'Year', 'Month']).size().rename('Records').reset_index()
    
    def printline(self, line):
        print(line + '\n')

In [453]:
op = OilProducers(datasetfolder + 'ProductionDataquery_2019.csv', '2019', True)
data = op.GetData()
op.OperatorFieldCount()
op.TopBottom5Overall()
op.UniqueDistrictMonthYear()
prod = op.ProducersInRange(350, 500, True)
prod

Oil Production Data for years:2019
Total Records: 104790, Non-Zero Records: 10627
Records > 300 bbl: 4830, 46% of Non-Zero Records
Total No. of Operators: 1459, Fields: 2742

Top and Bottom 5 Producers in all districts



,District,Year,Month,Operator,Name,No,Oil(BBL)
76585,8,2019,Jan,0,SPRABERRY (TREND AREA),85280300,22693221
1725,1,2019,Jan,0,EAGLEVILLE (EAGLE FORD-1),27135700,10738025
76319,8,2019,Jan,0,PHANTOM (WOLFCAMP),71052900,10376709
8505,2,2019,Jan,0,EAGLEVILLE (EAGLE FORD-2),27135750,10120581
74514,8,2019,Jan,1,"PIONEER NATURAL RES. USA, INC.",665748,5151051
48316,6,2019,Jan,0,QUITMAN (TRAVIS PEAK),73844820,301
7768,2,2019,Jan,1,"STAPLES, JAMES W. PROD. CO.",814691,301
20897,3,2019,Jan,0,"HOUSTON, SOUTH (MIOCENE)",42908110,301
84573,8A,2019,Jan,0,"NEW ERA, S. (GLORIETA)",65125210,301
33627,4,2019,Jan,0,JASPER-WEBB (YEGUA 3500),45837600,301


Unique District Month Year combination


,District,Year,Month,Records
0,1,2019,Jan,275
1,10,2019,Jan,184
2,2,2019,Jan,294
3,3,2019,Jan,525
4,4,2019,Jan,209
5,5,2019,Jan,87
6,6,2019,Jan,322
7,6E,2019,Jan,28
8,7B,2019,Jan,398
9,7C,2019,Jan,375


Oil Producers between 350 and 500: 612

Top 10 Producers with multiple wells: 
['BARNETT ENERGY, INC.', 'MARQUEE CORPORATION', 'JOSEY OIL COMPANY, LTD.', 'PITCOCK, INC.', 'TRIUM, LTD.', 'GREENWERKS OUTDOOR SERVICES LLC', 'HOME PETROLEUM CORP.', 'WEST', 'POWELL PROPERTIES, INC.', 'PHARAOH OIL & GAS, INC.']



,Name,Operator,Count,Oil(BBL)
248,"BARNETT ENERGY, INC.",1,3,451.333333
2272,MARQUEE CORPORATION,1,3,446.333333
1872,"JOSEY OIL COMPANY, LTD.",1,2,450.500000
2824,"PITCOCK, INC.",1,2,362.500000
3786,"TRIUM, LTD.",1,2,464.500000
1482,GREENWERKS OUTDOOR SERVICES LLC,1,2,376.500000
1680,HOME PETROLEUM CORP.,1,2,478.000000
4027,WEST,0,2,404.000000
2878,"POWELL PROPERTIES, INC.",1,2,445.500000
2794,"PHARAOH OIL & GAS, INC.",1,2,452.000000


In [454]:
op2 = OilProducers(datasetfolder + 'ProductionDataquery_2010_2019.csv', '2010_2019', True)
data2 = op.GetData()
op2.OperatorFieldCount()
op2.TopBottom5Overall()
prod2 = op2.ProducersInRange(350, 500, True)
prod2

Oil Production Data for years:2010_2019
Total Records: 4021690, Non-Zero Records: 1514805
Records > 300 bbl: 707425, 47% of Non-Zero Records
Total No. of Operators: 4446, Fields: 7197

Top and Bottom 5 Producers in all districts



,District,Year,Month,Operator,Name,No,Oil(BBL)
609944,8,2018,Aug,0,SPRABERRY (TREND AREA),85280300,26572404
616316,8,2018,Oct,0,SPRABERRY (TREND AREA),85280300,26218995
606753,8,2018,Jul,0,SPRABERRY (TREND AREA),85280300,26056159
613131,8,2018,Sep,0,SPRABERRY (TREND AREA),85280300,25821987
603555,8,2018,Jun,0,SPRABERRY (TREND AREA),85280300,25159209
1280985,08,2010,Jan,0,"HARPER, W. (PENNSYLVANIAN)",39184750,301
2164426,01,2012,Jul,1,"PRESSLY OIL INTEREST, INC.",676076,301
1404406,08,2013,Jul,0,"ANDECTOR, WEST (DEVONIAN)",2603250,301
1434963,08,2014,May,0,SHIPLEY (SILURIAN),83292625,301
3221483,04,2014,Jun,1,PRIME OPERATING COMPANY,677770,301


Oil Producers between 350 and 500: 3200

Top 10 Producers with multiple wells: 
['TEXOMA OIL TOOLS, INC.', 'WEST', 'SMITH RESOURCES, INC.', 'NORDSTRAND ENGINEERING, INC.', 'ROMA OIL AND GAS, INC.', 'PETROPLEX EQUIPMENT, INC.', 'PITCOCK, INC.', 'RABALAIS, F.W. INC.', 'HAGGERTON, TERRY', 'WES-MOR DRILLING, INC.']



,Name,Operator,Count,Oil(BBL)
10236,"TEXOMA OIL TOOLS, INC.",1,249,468.678715
11188,WEST,0,229,474.078603
9550,"SMITH RESOURCES, INC.",1,196,406.321429
7261,"NORDSTRAND ENGINEERING, INC.",1,194,462.489691
8824,"ROMA OIL AND GAS, INC.",1,192,484.317708
7876,"PETROPLEX EQUIPMENT, INC.",1,177,446.231638
7972,"PITCOCK, INC.",1,174,460.103448
8331,"RABALAIS, F.W. INC.",1,154,464.902597
4297,"HAGGERTON, TERRY",1,151,437.006623
11183,"WES-MOR DRILLING, INC.",1,150,476.766667


In [435]:
op3 = OilProducers(datasetfolder + 'ProductionDataquery_1997_2009.csv', '1997_2009', True)
data3 = op.GetData()
op3.OperatorFieldCount()
op3.TopBottom5Overall()
prod3 = op3.ProducersInRange(350, 500, True)
prod3

Oil Production Data for years: 1997_2009
Records > 300 bbl: 1213829, 20% of Total Records: 6332937
Total No. of Operators: 6713, Fields: 10271

Top and Bottom 5 Producers in all districts



,District,Year,Month,Operator,Name,No,Oil(BBL)
5347672,8A,1998,Aug,1,ALTURA ENERGY LTD.,15726,4088848
5351420,8A,1998,Oct,1,ALTURA ENERGY LTD.,15726,4027275
5360758,8A,1997,Mar,1,ALTURA ENERGY LTD.,15726,4006166
5338297,8A,1998,Mar,1,ALTURA ENERGY LTD.,15726,3994288
5342044,8A,1998,May,1,ALTURA ENERGY LTD.,15726,3994207
4542837,7C,1998,Jul,0,MIDSTATES (WINCHELL),61147700,301
2391823,04,2003,Sep,1,"JONES, R.W. & CO. II, LTD.",443926,301
3136798,06,2004,May,0,"HENDERSON, SOUTH (PETTIT -A-)",40361250,301
5551227,09,2006,Jul,1,FIELD PRODUCTION COMPANY,266910,301
4630304,08,2008,Feb,0,"PITZER, S. (DELAWARE)",71787500,301


Oil Producers between 350 and 500: 4336

Top 10 Producers with multiple wells: 
['CRESTON PETROLEUM CORP.', 'MARTIN, JIMMY INC', 'MCDANIEL OIL CO.', 'LADD OIL & GAS CORPORATION', 'MAYO, EDDIE', 'RAGARS OIL & GAS CO.', 'KENNEDY, R. R. PRODUCTION, INC.', 'CHARTER PETROLEUM COMPANY', 'CHP OIL & GAS COMPANY, L.C.', 'BANKHEAD OPERATING CORP.']



,Name,Operator,Count,Oil(BBL)
3653,CRESTON PETROLEUM CORP.,1,245,499.808163
9368,"MARTIN, JIMMY INC",1,244,469.450820
9579,MCDANIEL OIL CO.,1,228,493.070175
8328,LADD OIL & GAS CORPORATION,1,220,459.395455
9512,"MAYO, EDDIE",1,190,424.689474
12245,RAGARS OIL & GAS CO.,1,171,449.918129
7953,"KENNEDY, R. R. PRODUCTION, INC.",1,167,460.023952
2886,CHARTER PETROLEUM COMPANY,1,167,439.329341
2989,"CHP OIL & GAS COMPANY, L.C.",1,166,467.210843
1014,BANKHEAD OPERATING CORP.,1,158,420.316456
